In [2]:
!pip uninstall -y openai
!pip install openai==0.27.0 jsonschema


Found existing installation: openai 0.27.0
Uninstalling openai-0.27.0:
  Successfully uninstalled openai-0.27.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 946.9/946.9 kB 15.6 MB/s eta 0:00:00


In [23]:
import os
import json
from jsonschema import validate
from openai import OpenAI


In [ ]:
os.environ["GROQ_API_KEY"] = "groq_api_key"
BASE_URL = "https://api.groq.com/openai/v1"
MODEL = "llama-3.1-8b-instant"


In [25]:
client = OpenAI(base_url=BASE_URL, api_key=os.environ["GROQ_API_KEY"])


In [26]:
class ConversationManager:
    def __init__(self, client, model, k=3):
        self.client = client
        self.model = model
        self.k = k
        self.history = []
        self.run_count = 0

    def add_message(self, role, content):
        self.history.append({"role": role, "content": content})

    def summarize_history(self):
        if len(self.history) < self.k:
            return self.history

        to_summarize = self.history[:-self.k]
        response = self.client.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": "Summarize the conversation concisely."},
                {"role": "user", "content": json.dumps(to_summarize)}
            ]
        )
        summary = response.choices[0].message.content
        self.history = [{"role": "system", "content": f"[Summary]: {summary}"}] + self.history[-self.k:]
        return summary

    def step(self, role, content):
        self.add_message(role, content)
        self.run_count += 1
        if self.run_count % self.k == 0:
            self.summarize_history()

        response = self.client.chat.completions.create(
            model=self.model,
            messages=self.history
        )
        reply = response.choices[0].message.content
        self.add_message("assistant", reply)
        return reply


In [27]:
def print_chat_history(history):
    print("===== Chat History =====")
    for msg in history:
        role = msg["role"].capitalize()
        content = msg["content"]
        if role == "System" and "Summary" in content:
            print(f"[{role}]: {content}")
        else:
            print(f"{role}: {content}")
    print("========================\n")


In [28]:
conv_manager = ConversationManager(client, MODEL, k=2)

print(conv_manager.step("user", "Hi, can you tell me about Python?"))
print_chat_history(conv_manager.history)

print(conv_manager.step("user", "What are its main uses?"))
print_chat_history(conv_manager.history)

print(conv_manager.step("user", "How is Python different from Java?"))
print_chat_history(conv_manager.history)


Python is a high-level, interpreted programming language that is widely used for various purposes such as web development, scientific computing, data analysis, artificial intelligence, and more. Here are some key features and uses of Python:

**Key Features:**

1. **Easy to Learn:** Python has a simple syntax and is relatively easy to learn, making it a great language for beginners.
2. **High-Level Language:** Python is a high-level language, which means it abstracts away many low-level details, allowing developers to focus on the logic of the program rather than the details of the computer.
3. **Interpreted Language:** Python code is interpreted rather than compiled, which means that the code is executed line by line rather than all at once.
4. **Dynamic Typing:** Python is dynamically typed, which means that variables do not need to be declared before they are used.
5. **Large Standard Library:** Python has a vast collection of libraries and modules that can be used to perform a wide

In [29]:
schema = {
    "type": "object",
    "properties": {
        "name": {"type": ["string", "null"]},
        "email": {"type": ["string", "null"]},
        "phone": {"type": ["string", "null"]},
        "location": {"type": ["string", "null"]},
        "age": {"type": ["integer", "null"]}
    },
    "required": ["name", "email", "phone", "location", "age"]
}

def extract_info(chat_text):
    response = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": "Extract name, email, phone, location, age as JSON."},
            {"role": "user", "content": chat_text}
        ],
        response_format={"type": "json_object"}
    )
    data = json.loads(response.choices[0].message.content)
    validate(instance=data, schema=schema)
    return data


In [30]:
samples = [
    "Hi, my name is Rahul. I live in Delhi. My email is rahul123@gmail.com and I’m 22.",
    "Hello, I’m Ananya from Mumbai, 25 years old. You can reach me at ananya99@yahoo.com",
    "This is Arjun, my phone is 9876543210, I stay in Bangalore."
]

for s in samples:
    print("Chat:", s)
    print("Extracted JSON:", extract_info(s))
    print()


Chat: Hi, my name is Rahul. I live in Delhi. My email is rahul123@gmail.com and I’m 22.
Extracted JSON: {'name': 'Rahul', 'email': 'rahul123@gmail.com', 'phone': None, 'location': 'Delhi', 'age': 22}

Chat: Hello, I’m Ananya from Mumbai, 25 years old. You can reach me at ananya99@yahoo.com
Extracted JSON: {'name': 'Ananya', 'email': 'ananya99@yahoo.com', 'phone': None, 'location': 'Mumbai', 'age': 25}

Chat: This is Arjun, my phone is 9876543210, I stay in Bangalore.
Extracted JSON: {'name': 'Arjun', 'email': None, 'phone': '9876543210', 'location': 'Bangalore', 'age': None}

